In [4]:
import random
from PIL import Image, ImageDraw, ImageFont
class Space:
    '''defines the problem space
    
        ARGS: 
        - height and width of the space 
        - num_hospitals - number of hospitals
        - houses and hospitals  - declared as sets
    '''
    def __init__(self, height, width, num_hospitals) -> None:
        #class constructor
        self.height = height
        self.width = width
        self.num_hospitals = num_hospitals
        self.houses =set()
        self.hospitals = set()
    #add a house method
    def add_house(self, row, col):
        '''row and col specifies coordinate '''
        self.houses.add((row,col))
    
    #method to find out available spaces so we do not add a house /hospital to a used up space
    def available_space(self):
        candidates = set(
            (row, col) #this has all rows and columns
            #run a for loop along the height to get the available rows
            for row in range(self.height)
            #another for loop along the width to get the columns
            for col in range(self.width)
        )
        
        #now remove house and hospital from the candidates
        for house in self.houses:
            candidates.remove(house)
        for hospital in self.hospitals:
            candidates.remove(hospital)
        return candidates # this is now a set of available spaces
    
    #get the cost method that defines the distance between the house and hospital
    def get_cost(self, hospitals):
        cost =0
        for house in self.houses:
            # 0 means the row value / x axis  and 1 is y axis use abslute to get the positive value and wrap it around min function to get min cost
             #to access the individual hospital instead of the passed set use the for loop below
            cost += min(abs(house[0] - hospital[0]) + abs(house[1] - hospital[1]) for hospital in hospitals)
        return cost
        
    #get the neighbours
    def get_neighbours(self, row,col):
        
        #list of neighbours
        candidates = [
            
            (row -1, col), #top neighbour
            (row +1, col), #bottom neighbour
            (row, col-1), # left neighbour
            (row, col+1) #right neighbour
        ]
        #list of candidates that cannot be neighbours i.e. occupied or out of the search sepace
        neighbours =[]
            #iterate though rows and columns and see if row and column belongs to house or hospital continue
        for r,c in candidates:
            if (r, c) in self.houses or (r, c) in self.hospitals:
                continue
            #if its in the the search space
            if 0 <= r < self.height and 0 <= c <self.width:
                neighbours.append((r, c))
        return neighbours
        
    #generate output image to graphically visualise the output
    def output_image(self,filename):
        
        cell_size = 100
        cell_border =2
        cost_size = 40
        padding = 10

        #create an RGBA image
        img = Image.new("RGBA", 
                        (self.width * cell_size, self.height * cell_size + cost_size+padding*2),
                        "white")
        
        #load images and resize as per the cell size
        house = Image.open("assets/images/House.png").resize((cell_size, cell_size))
        hospital = Image.open("assets/images/Hospital.png").resize((cell_size, cell_size))
        font = ImageFont.truetype("assets/fonts/OpenSans-Regular.ttf",30)
        
        #draw the image
        draw = ImageDraw.Draw(img)
        
        #iterate through space and place a red selection rectangle
        for i in range(self.height):
            for j in range(self.width):
                rect = [
                    (j *cell_size + cell_border, i * cell_size + cell_border), 
                    ((j+1) * cell_size - cell_border, (i +1) * cell_size -cell_border)
                    
                ]
                draw.rectangle(rect, fill="Black")
                #if coordinate belows to the house then copy the house
                if (i, j) in self.houses:
                    img.paste(house, rect[0], house)
                if (i, j) in self.hospitals:
                    img.paste(hospital, rect[0], hospital)
        draw.rectangle(
            (0, self.height*cell_size, self.width *cell_size,
             self.height *cell_size + cost_size+padding * 2),
            "black"
             )
       #draw the text that will print in white font and save the file
        draw.text(
            (padding,self.height*cell_size + padding ), 
            f"Cost: {self.get_cost(self.hospitals)}",
            fill="white",
            font =font
        )
        img.save(filename) 
    
    #hill climbimog method
    #maximum controls iterations, image prefix to track multiple output images, log is the optimisation progress , 
    def hill_climb(self, maximum=None, image_prefix =None, log=False):
        count =0
        self.hospitals = set()
        #add hospitals available randomly
        for i in range(self.num_hospitals):
            self.hospitals.add(random.choice(list(self.available_space())))
        if log:
            print("Initial State: Cost", self.get_cost(self.hospitals))
        if image_prefix:
            self.output_image(f"{image_prefix}{str(count).zfill(3)}.png") #zfill fills filename with 3 zeros i.e. we can gett 1000 image 000 to 999
        
        while maximum is None or count <maximum:
            count +=1
            best_neighbours = []
            best_neighbour_cost =None
            
            for hospital in self.hospitals:
                for replacement in self.get_neighbours(*hospital):
                    neighbour = self.hospitals.copy()
                    
                    neighbour.remove(hospital)
                    neighbour.add(replacement)
                    
                    cost = self.get_cost(neighbour)
                    if best_neighbour_cost is None or cost < best_neighbour_cost:
                        best_neighbour_cost = cost
                        best_neighbours = [neighbour]
                    elif best_neighbour_cost == cost:
                        best_neighbours.append(neighbour)
            if best_neighbour_cost >= self.get_cost (self.hospitals):
                return self.hospitals # nothing will happen
            else:
                if log:
                    print(f"Found Better Neighbour: cost {best_neighbour_cost}")
                self.hospitals = random.choice(best_neighbours)
            if image_prefix:
                self.output_image(f"{image_prefix}{str(count).zfill(3)}.png")     
            
s = Space(height =6, width=12, num_hospitals=2)

for i in range(5):
    s.add_house(random.randrange(s.height), random.randrange(s.width))

hospitals = s.hill_climb(image_prefix="hospitals", log=True )        
                    
                    
                
        
            
        
        
        
        

Initial State: Cost 31
Found Better Neighbour: cost 28
Found Better Neighbour: cost 25
Found Better Neighbour: cost 23
Found Better Neighbour: cost 21
Found Better Neighbour: cost 20
Found Better Neighbour: cost 19
Found Better Neighbour: cost 16
Found Better Neighbour: cost 15
Found Better Neighbour: cost 14
Found Better Neighbour: cost 13


In [5]:
#Running the iteration

s = Space(height =6, width=12, num_hospitals=2)

for i in range(5):
    s.add_house(random.randrange(s.height), random.randrange(s.width))

hospitals = s.hill_climb(image_prefix="hospitals", log=True )
    

Initial State: Cost 32
Found Better Neighbour: cost 28
Found Better Neighbour: cost 24
Found Better Neighbour: cost 22
Found Better Neighbour: cost 20
Found Better Neighbour: cost 18
Found Better Neighbour: cost 16
Found Better Neighbour: cost 14
Found Better Neighbour: cost 13
Found Better Neighbour: cost 12
Found Better Neighbour: cost 11
